Считывание и инициализация.

In [3]:
import warnings  
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.stats import skew
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier


X_train = pd.read_csv('Train.csv')
X_test = pd.read_csv('Test.csv')


print(X_train['street_id'].nunique())
print(X_train['area'].nunique())


672
112


Работа с фичами.

In [4]:
test_index=X_test['id'].values
print(test_index)
y=X_train.price
X_train.drop(['price'], axis=1, inplace=True)
def featureProcessing(df):
    df.drop(['id'], axis=1, inplace=True)
    df['Y']=df['date'].apply(lambda x: int(x.split('-')[0]))
    df['M']=df['date'].apply(lambda x: int(x.split('-')[1]))
    df.drop(['date'], axis=1, inplace=True)
    return df
X_train=featureProcessing(X_train)
X_test=featureProcessing(X_test)

[100000 100001 100002 ... 199997 199998 199999]


Обучение модели

In [22]:
from catboost import CatBoostClassifier
model_XGB = CatBoostClassifier(iterations=2,
                          learning_rate=1,
                          depth=2)
#n_estimators=10000, learning_rate=0.055
print(X_train.values[0])
print(y.values)
model_XGB.fit(X=X_train.values[0], y=np.log1p(y.values))
#,cat_features=['area','street_id']
ensemble_preds = model_XGB.predict(X_test.values[0])

[5.110e+02 1.000e+00 5.000e+00 6.700e+01 2.000e+00 1.000e+00       nan
 1.000e+00 1.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 2.011e+03 1.000e+00]
[18473000  4157000  4530000 ...  4996000  1258000  3507000]


CatboostError: Length of label=100000 and length of data=24 is different.

In [4]:
preds = np.expm1(ensemble_preds)

output = pd.DataFrame({'id': test_index,
                      'price': preds})

output.to_csv('submission.csv', index=False)

NameError: name 'ensemble_preds' is not defined